# Week 8: Word Vectors

In this weeks sesssion we are going to be looking at word vectors. Word vectors (or *word embeddings*) are pretrained numerical representations of words, within a high-dimensional vector space. The number of dimensions for our word vectors is arbitrary and can range anywhere from 50 to 300 dimensions. 

Word vectors are calculated from very large datasets of texts, with the goal of words that are similiar being close to each other in vector space, and words being dissimilar being far away in vector space. After processing a vast amount of data, we end up with a unique vector for every word in the corpus. This gives us feature representations of words (that unlike other representations of words we have seen like one-hot, bag of words or TF-IDF) encode a representation that captures the meaning of the word. 

As these word vectors are **numerical representations**, we can perform mathematical functions on them to do some interesting (and revealing) insights into what kind of data and biases these models contain. 

First lets do some imports:

In [ ]:
import torch
import torch.nn as nn
import torchtext.vocab as vocab

#### Download word vectors

Here we are going to load our set of word vectors using the torchtext library. Here we are downloading the [GloVe pretrained word embeddings](https://nlp.stanford.edu/projects/glove/) trained a data dump of Wikipedia from 2014. There are [other pretrained word embeddings](https://torchtext.readthedocs.io/en/latest/vocab.html#pretrained-word-embeddings) available in torchtext. You can try loading in other ones later and see how that effects results. 

This download is about 1GB. You should **run this before the class**. If you haven't done this before the class for whatever reason and it is taking too long to download in class then kill the cell (or restart the kernel) and instead use the function in the cell following the next one. 

In [ ]:
word_vectors = vocab.GloVe(name="6B",dim=100) 

##### (Alternative) load a sub-sample of word vectors

If the previous cell is taking too long to download, you can uncomment this line to load in a sample of the top 30K word vectors from GloVe to use for this exercise:

In [ ]:
#word_vectors = vocab.Vectors(name = '../data/glove.6B.100d.top30k.txt')

Lets take a look of one of our word vectors. It looks like a big list of numbers!

In [ ]:
word_vectors['dog']

Lets take a look at another one:

In [ ]:
word_vectors['cat']

On their own, these word vectors are not particularly meaningful. No person looking a this would not be able to make sense of it's meaning. 

Where word vectors become powerful is when we make comparisons between them. We can use the [cosine similarity](https://pytorch.org/docs/stable/generated/torch.nn.CosineSimilarity.html) function in PyTorch to get a measure of similarity between our two vectors. 

As this is a similarity measurement, the higher the value the most similar. 1 is the highest value we can get and 0 is the lowest value. Lets compare our word vectors:

In [ ]:
cosine_sim = nn.CosineSimilarity(dim=0)

similarity = cosine_sim(word_vectors['dog'], word_vectors['dog'])
print(f'The words dog and dog have a cosine similiarity of {similarity.item():3f}')

similarity = cosine_sim(word_vectors['dog'], word_vectors['cat'])
print(f'The words dog and cat have a cosine similiarity of {similarity.item():3f}')

Now lets compare some more words:

In [ ]:
similarity = cosine_sim(word_vectors['dog'], word_vectors['fox'])
print(f'The words dog and fox have a cosine similiarity of {similarity.item():3f}')
similarity = cosine_sim( word_vectors['cat'], word_vectors['fox'])
print(f'The words cat and fox have a cosine similiarity of {similarity.item():3f}')

Foxes are in the canine family so this is accurate! 

Now lets compare London to some cities around the world:

In [ ]:
similarity = cosine_sim(word_vectors['london'], word_vectors['paris'])
print(f'The words london and paris have a cosine similiarity of {similarity.item():3f}')
similarity = cosine_sim(word_vectors['london'], word_vectors['madrid'])
print(f'The words london and madrid have a cosine similiarity of {similarity.item():3f}')
similarity = cosine_sim(word_vectors['london'], word_vectors['beirut'])
print(f'The words london and beirut have a cosine similiarity of {similarity.item():3f}')
similarity = cosine_sim(word_vectors['london'], word_vectors['beijing'])
print(f'The words london and beijing have a cosine similiarity of {similarity.item():3f}')

And cities in the UK:

In [ ]:
similarity = cosine_sim(word_vectors['london'], word_vectors['edinburgh'])
print(f'The words london and edinburgh have a cosine similiarity of {similarity.item():3f}')
similarity = cosine_sim(word_vectors['london'], word_vectors['glasgow'])
print(f'The words london and glasgow have a cosine similiarity of {similarity.item():3f}')
similarity = cosine_sim(word_vectors['glasgow'], word_vectors['edinburgh'])
print(f'The words glasgow and edinburgh have a cosine similiarity of {similarity.item():3f}')

And cities in Ireland:

In [ ]:
similarity = cosine_sim(word_vectors['london'], word_vectors['dublin'])
print(f'The words london and dublin have a cosine similiarity of {similarity.item():3f}')
similarity = cosine_sim(word_vectors['london'], word_vectors['belfast'])
print(f'The words london and belfast have a cosine similiarity of {similarity.item():3f}')
similarity = cosine_sim(word_vectors['dublin'], word_vectors['belfast'])
print(f'The words dublin and belfast have a cosine similiarity of {similarity.item():3f}')

#### Measure distances on your own words

Try putting your own words in here to see the distance scores:

In [ ]:
word1 = ''
word2 = ''
similarity = cosine_sim(word_vectors[word1], word_vectors[word2])
print(f'These words have a distance of: {similarity.item():3f}')

### Finding closest words

The following function will let us look for the closest words in vector space to a target word. The following function calculates this using the [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) instead of the Cosine Similarity. 

In [ ]:
#This function is sourced from: https://www.cs.toronto.edu/~lczhang/321/lec/glove_notes.html
def print_closest_words(vec, n=5):
    dists = torch.norm(word_vectors.vectors - vec, dim=1)     # compute distances to all words
    lst = sorted(enumerate(dists.numpy()), key=lambda x: x[1]) # sort by distance
    for idx, difference in lst[1:n+1]: 					       # take the top n
        print(word_vectors.itos[idx], difference)

In [ ]:
print_closest_words(word_vectors["dog"], n=10)

In [ ]:
print_closest_words(word_vectors["london"], n=10)

In [ ]:
print_closest_words(word_vectors["camberwell"], n=10)

In [ ]:
print_closest_words(word_vectors["potato"], n=10)

In [ ]:
print_closest_words(word_vectors["doctor"], n=10)

#### Try putting your own words into this function:

In [ ]:
my_word = 'word'
print_closest_words(word_vectors[my_word], n=10)

#### Doing arithmetic on word vectors

We can do arithmetic on word vectors to create new vectors:

In [ ]:
new_word_vector = word_vectors['king'] - word_vectors['man'] + word_vectors['woman']
new_word_vector

Once again, this is not very interpretable. But we can use mathematical functions to learn more about the new word vectors we have created: 

In [ ]:
similarity = cosine_sim(new_word_vector, word_vectors['man'])
print(f'Our new vector has a cosine similarity of {similarity.item():3f} to the word man')
similarity = cosine_sim(new_word_vector, word_vectors['woman'])
print(f'Our new vector has a cosine similarity of {similarity.item():3f} to the word woman')

And we can use our our search function to find the closest word vectors to our new word in vector space:

In [ ]:
print_closest_words(new_word_vector, n=10)

### Investigating bias in word vectors

Now lets use these tools to see expose the biases encoded in word vectors. 

If we subtract man from the word doctor, and add the vector for woman, the closest word vectors are:

In [ ]:
print_closest_words(word_vectors['doctor'] - word_vectors['man'] + word_vectors['woman'])

However when we subtract woman from doctor and add man, we do not get the same effect:

In [ ]:
print_closest_words(word_vectors['doctor'] - word_vectors['woman'] + word_vectors['man'])

However when we do the same thing with the word nurse, then we do get the word doctor:

In [ ]:
print_closest_words(word_vectors['nurse'] - word_vectors['woman'] + word_vectors['man'])

#### Try investigating your own words for bias:

Plug in different words here and investigate your own kinds of bias. It does not have to be gender bias, it could be racial, class, sexuality, disability or other. 

In [ ]:
original_word = 'academic'
negative_word = 'man'
positive_word = 'woman'
new_word_vector = word_vectors[original_word] - word_vectors[negative_word] + word_vectors[positive_word]
print_closest_words(new_word_vector)

Have a go at [loading in](#download-word-vectors) some of the [different word vectors available on torchtext](https://torchtext.readthedocs.io/en/latest/vocab.html#pretrained-word-embeddings), or using a different dimensionality for the GloVe vectors and re-run the cells in this notebook. How does that impact the results? (You may want to make a copy of this notebook to make a side-by-side comparison)